In [1]:
from S3DogDataset import *
import pandas as pd
from torch.utils.data import DataLoader, RandomSampler
from torchvision import models
from tqdm import tqdm
import time

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
train_dataset = S3DogDataset(file_ref_csv='Labels/train_labels.csv', bucket_name = "sagemaker-team11-stanford-dogs")
test_dataset = S3DogDataset(file_ref_csv='Labels/test_labels.csv', bucket_name = "sagemaker-team11-stanford-dogs")

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True, num_workers= 24)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True, pin_memory=True, num_workers= 24)

train_sampler = RandomSampler(train_dataset, replacement=True, num_samples=2400)
test_sampler = RandomSampler(test_dataset, replacement=True, num_samples=1200)

train_sample_loader = DataLoader(train_dataset, batch_size=32, sampler=train_sampler, pin_memory=True, num_workers= 24)
test_sample_loader = DataLoader(test_dataset, batch_size=32, sampler=test_sampler, pin_memory=True, num_workers= 24)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [4]:
num_classes = 120

# load mobilenet_v3_large
mobilenet_v3_large = models.mobilenet_v3_large(weights="IMAGENET1K_V2")

for param in mobilenet_v3_large.parameters():
    param.requires_grad = False

num_features = mobilenet_v3_large.classifier[3].in_features
mobilenet_v3_large.classifier[3] = torch.nn.Linear(num_features, num_classes)

# load mobilenet_v3_small
mobilenet_v3_small = models.mobilenet_v3_small(weights="IMAGENET1K_V1")
            
for param in mobilenet_v3_small.parameters():
    param.requires_grad = False

num_features = mobilenet_v3_small.classifier[3].in_features
mobilenet_v3_small.classifier[3] = torch.nn.Linear(num_features, num_classes)

# load mnasnet0_5
mnasnet0_5 = models.mnasnet0_5(weights="IMAGENET1K_V1")
            
for param in mnasnet0_5.parameters():
    param.requires_grad = False

num_features = mnasnet0_5.classifier[1].in_features
mnasnet0_5.classifier[1] = torch.nn.Linear(num_features, num_classes)

# load shufflenet_v2_x0_5
shufflenet_v2_x0_5 = models.shufflenet_v2_x0_5(weights="IMAGENET1K_V1")
            
for param in shufflenet_v2_x0_5.parameters():
    param.requires_grad = False

num_features = shufflenet_v2_x0_5.fc.in_features
shufflenet_v2_x0_5.fc = torch.nn.Linear(num_features, num_classes)

# load shufflenet_v2_x1_0
shufflenet_v2_x1_0 = models.shufflenet_v2_x1_0(weights="IMAGENET1K_V1")
            
for param in shufflenet_v2_x1_0.parameters():
    param.requires_grad = False

num_features = shufflenet_v2_x1_0.fc.in_features 
shufflenet_v2_x1_0.fc = torch.nn.Linear(num_features, num_classes)

# specify model lists and names
# model_list = [mobilenet_v3_small, mobilenet_v3_large, mnasnet0_5, shufflenet_v2_x0_5,shufflenet_v2_x1_0]
# model_name_list = ['mobilenet_v3_small.pth','mobilenet_v3_large.pth','mnasnet0_5.pth','shufflenet_v2_x0_5.pth','shufflenet_v2_x1_0.pth']

In [4]:
# load microsoft's tiny_vit
import timm

#from tiny_vit import tiny_vit_5m_224, tiny_vit_11m_224, tiny_vit_21m_224
from tiny_vit import tiny_vit_5m_224

num_classes = 120

tiny_vit_5m = tiny_vit_5m_224(pretrained=True)
tiny_vit_5m.head = torch.nn.Linear(tiny_vit_5m.head.in_features, num_classes)

# model_list = [tiny_vit_5m]
# model_name_list = ['tiny_vit_5m.pth']

In [11]:
model_list = [mobilenet_v3_small, mobilenet_v3_large, mnasnet0_5, shufflenet_v2_x0_5,shufflenet_v2_x1_0, tiny_vit_5m]
model_name_list = ['mobilenet_v3_small.pth','mobilenet_v3_large.pth','mnasnet0_5.pth','shufflenet_v2_x0_5.pth','shufflenet_v2_x1_0.pth', 'tiny_vit_5m.pth']

In [12]:
train_loader = train_sample_loader
test_loader = test_sample_loader

for idx, (model, model_name) in enumerate(zip(model_list, model_name_list)):

    num_epochs = 10
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    criterion = torch.nn.CrossEntropyLoss()

    if torch.cuda.is_available():
        device = torch.device('cuda') 
    else:
        device = torch.device('cpu')

    for param in model.parameters():
        param.requires_grad = True

    model = model.to(device)

    results_df = pd.DataFrame(columns=['Model', 'Epoch', 'Accuracy', 'Train Loss', 'Val Loss', 'Epoch Time'])

    for epoch in range(num_epochs):
        epoch_start = time.time()
        
        model.train()
        for batch_idx, (images, labels) in enumerate(tqdm(train_loader, desc='Train Loop')):
            images = images.to(device)
            labels = labels.to(device)
            optimizer.zero_grad(set_to_none=True)
            outputs = model(images)
            train_loss = criterion(outputs, labels)
            train_loss.backward()
            optimizer.step()
                        
        model.eval()
        with torch.no_grad():
            correct = 0
            total = 0
            for batch_idx, (images, labels) in enumerate(tqdm(test_loader, desc='Validation Loop')):
                images = images.to(device)
                labels = labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                val_loss = criterion(outputs, labels)

        print(f'Model: {model_name}, Epoch: {epoch+1}, Accuracy: {100 * correct / total}, Train Loss: {train_loss.item()}, Val Loss: {val_loss.item()}, Epoch Time: {time.time()-epoch_start}')
        
        if not os.path.exists('Results'):
            os.makedirs('Results')

        results_df.loc[len(results_df)] = [model_name, (epoch+1), (100 * correct / total), train_loss.item(), val_loss.item(), (time.time()-epoch_start)]
        results_df.to_csv(f'Results/{model_name}.csv', index=False)
            
        if not os.path.exists('Models'):
            os.makedirs('Models')
                
        torch.save(model.state_dict(), f'Models/{model_name}')
        print(f'Model: {model_name} saved.')
     

Validation Loop: 100%|██████████| 38/38 [00:21<00:00,  1.75it/s]


Model: tiny_vit_11m.pth, Epoch: 1, Accuracy: 7.833333333333333, Train Loss: 4.565139293670654, Val Loss: 4.499314308166504, Epoch Time: 88.86070919036865
Model: tiny_vit_11m.pth saved.


Validation Loop: 100%|██████████| 38/38 [00:21<00:00,  1.80it/s]


Model: tiny_vit_11m.pth, Epoch: 2, Accuracy: 26.5, Train Loss: 4.214290142059326, Val Loss: 4.242842197418213, Epoch Time: 91.90541315078735
Model: tiny_vit_11m.pth saved.


Validation Loop: 100%|██████████| 38/38 [00:40<00:00,  1.06s/it]


...

Validation Loop: 100%|██████████| 38/38 [00:21<00:00,  1.77it/s]


Model: tiny_vit_21m_224.pth, Epoch: 4, Accuracy: 47.0, Train Loss: 3.165919303894043, Val Loss: 3.163090944290161, Epoch Time: 94.98359394073486
Model: tiny_vit_21m_224.pth saved.


Validation Loop: 100%|██████████| 38/38 [00:21<00:00,  1.77it/s]


Model: tiny_vit_21m_224.pth, Epoch: 5, Accuracy: 54.25, Train Loss: 2.738288640975952, Val Loss: 3.163104772567749, Epoch Time: 95.49677300453186
Model: tiny_vit_21m_224.pth saved.


Train Loop:  20%|██        | 15/75 [00:27<00:56,  1.07it/s]

- Trained on ml.c5.9xlarge
- Bottleneck in training time is the num_of_workers available to support data loading.
- Batchsize = 32
- Optimizer = Adam 
- lr = .0001
- Trained with a random sampler to increase observability of metrics 
- Tuned 5 of the smallest pretrained models available via Pytorch
- 10 epochs, should maybe increase, because the first model appears like it had more to go, based on train and val loss
- Benchmarking 'light-weight' pretrained models on the stanford dogs dataset using AWS and PyTorch

- continued exploring with tiny vit, https://arxiv.org/abs/2207.10666 